# Scraping: récupération de la page d'accueil du journal Le Soir

Dans ce notebook, nous créons un robot qui va ouvrir la page d'accueil du site du journal [Le Soir](https://www.lesoir.be/) et récupérer le titre de tous les articles du jour et les stocker dans un fichier csv.

## Imports

In [3]:
import os
import re
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Récupération de tous les articles de la page d'accueil



In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

articles = []

root_url = f"https://www.rtbf.be"
response = requests.get(root_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
for link in soup.find_all("a", {"class":"stretched-link outline-none"}):
    title = link.text.strip()
    url = root_url + link.get('href')
    articles.append([url, title])

In [5]:
# Affichage du nombre d'articles récupérés
len(articles)

83

In [6]:
# Affichage des 10 premières entrées
articles[:10]

[['https://www.rtbf.be/elections-2024/communales/resultats/moorslede-8890',
  'Moorslede'],
 ['https://www.rtbf.be/elections-2024/communales/resultats/beersel-1650',
  'Beersel'],
 ['https://www.rtbf.be/elections-2024/communales/resultats/gand-9000', 'Gand'],
 ['https://www.rtbf.be/article/direct-elections-communales-et-provinciales-suivez-minute-par-minute-les-elections-de-ce-dimanche-13-octobre-11447930',
  'Direct – Élections communales et provinciales\xa0: le PS reste en tête à Mons, Charleroi, Liège et Bruxelles, le dépouillement continue'],
 ['https://www.rtbf.be/article/bart-de-wever-grand-vainqueur-a-anvers-rome-a-gagne-11448422',
  'Bart De Wever, grand vainqueur à Anvers\xa0: "Rome a gagné\xa0!"'],
 ['https://www.rtbf.be/article/elections-communales-a-chievres-ecolo-paie-le-prix-de-la-crise-des-pfas-11448404',
  'Elections communales à Chièvres\xa0: Ecolo paie le prix de la crise des PFAS'],
 ['https://www.rtbf.be/article/resultats-des-elections-analyses-projections-interview

## Création d'un dataframe avec les liens et les titres des articles


In [7]:
df = pd.DataFrame(articles, columns=['link', 'title'])
df

,link,title
0,https://www.rtbf.be/elections-2024/communales/...,Moorslede
1,https://www.rtbf.be/elections-2024/communales/...,Beersel
2,https://www.rtbf.be/elections-2024/communales/...,Gand
3,https://www.rtbf.be/article/direct-elections-c...,Direct – Élections communales et provinciales ...
4,https://www.rtbf.be/article/bart-de-wever-gran...,"Bart De Wever, grand vainqueur à Anvers : ""Rom..."
...,...,...
78,https://www.rtbf.behttps://auvio.rtbf.be/media...,Météo 13h30
79,https://www.rtbf.behttps://auvio.rtbf.be/media...,JT 13h
80,https://www.rtbf.behttps://auvio.rtbf.be/media...,Cultes et relais
81,https://www.rtbf.behttps://auvio.rtbf.be/media...,Un si grand soleil


In [8]:
# Sauvegarde du dataframe dans un fichier csv
df.to_csv(f"../data/rtbf_{time.strftime('%Y%m%d')}.csv", index=False)

## Téléchargement d'un article et affichage du texte

In [9]:
# Récupération du premier lien du dataframe
article_url = df.iloc[0][0]

# Ouverture de la page
response = requests.get(article_url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
# Parsing du contenu
title = soup.find("h1").get_text(separator=' ')

paragraphs = []
for paragraph in soup.find_all("p", attrs={"class": None}):
    paragraphs.append(paragraph.get_text(separator=' ').strip())

content = "\n".join(paragraphs)
print(title)
print("==================================")
print(content)

C:\Users\tdeganck\AppData\Local\Temp\ipykernel_30196\3125040403.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  article_url = df.iloc[0][0]


Résultats élections communales 2024 Moorslede - 8890
Copyright ©  2024  RTBF


### Nettoyage du texte à l'aide d'expressions régulières

In [10]:
# Suppression de tout ce qui se trouve entre parenthèses
clean_content = re.sub("\([^\)]+\)", " ", content)
# Suppression des espaces multiples
clean_content = re.sub("\s+", " ", clean_content)

print(content)
print("==================================")
print(clean_content)


Copyright ©  2024  RTBF
Copyright © 2024 RTBF


### Création d'un fichier avec le contenu de l'article


In [ ]:
with open("../data/rtbf_example.txt", "w") as writer:
    writer.write(f"{title}\n\n{clean_content}")

## Pour en savoir plus

- Le web scraping avec Python: https://realpython.com/beautiful-soup-web-scraper-python/
- Tutoriel sur les expressions régulières: https://www.w3schools.com/python/python_regex.asp